In [1]:
from datasets import load_dataset,load_from_disk
from sentence_transformers import SentenceTransformer, util
from pyarrow import json
from pathlib import Path
import pyarrow as pa
from datasets import load_dataset,load_from_disk
import numpy as np

In [2]:
experiments_root = Path('../resources/preprocessed_data/wikilarge_original')

In [ ]:
data = load_from_disk(experiments_root)

In [ ]:
data

In [ ]:
#data = data.select(range(1000))

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')
def calculate_similarity(row):
    embeddings1 = model.encode(row['original_text'], convert_to_tensor=True)
    embeddings2 = model.encode(row['simple_text'], convert_to_tensor=True)
    cosine_scores = util.cos_sim(embeddings1, embeddings2)
    row['similarity'] = cosine_scores[0][0].detach().cpu().item()
    print(row)
    return row

In [ ]:
data = data.map(calculate_similarity)

In [ ]:
data.save_to_disk('../resources/preprocessed_data/wikilarge_sim')

In [ ]:
data

Experiments with different values of similarity

In [2]:
data = load_from_disk('../resources/preprocessed_data/wikilarge_sim')

In [3]:
data

Dataset({
    features: ['original_text', 'simple_text', 'original_text_preprocessed', 'WLR', 'CLR', 'LR', 'DTDR', 'WRR', 'LMFMR', 'similarity'],
    num_rows: 296402
})

In [4]:
data_df = data.to_pandas()

In [5]:
data_df

,original_text,simple_text,original_text_preprocessed,WLR,CLR,LR,DTDR,WRR,LMFMR,similarity
0,There is manuscript evidence that Austen conti...,There is some proof that Austen continued to w...,,0.89,0.91,0.68,0.86,0.92,0.69,0.937799
1,"In a remarkable comparative analysis , Mandaea...",Mandaean scholar Säve-Söderberg showed that Ma...,,0.75,0.71,0.81,1.00,0.99,1.47,0.972845
2,"Before Persephone was released to Hermes , who...",When Demeter went to the Underworld to rescue ...,,0.96,0.93,0.51,0.44,1.01,1.60,0.780261
3,Cogeneration plants are commonly found in dist...,Cogeneration plants are commonly found in dist...,,1.03,1.01,0.98,1.00,1.01,1.11,0.997548
4,"Geneva -LRB- , ; , ; , ; ; -RRB- is the second...",The city 's main newspaper is the Tribune de G...,,0.89,0.77,0.33,1.50,0.97,7.17,0.008508
...,...,...,...,...,...,...,...,...,...,...
296397,These structures form at the high-pressure dep...,These structures form at the high-pressure dep...,,1.05,1.03,0.96,1.20,1.00,1.23,0.993901
296398,Saint-Maixant is a commune in the Gironde depa...,"Saint-Maixant , Gironde is a commune . It is f...",,1.71,1.42,0.70,1.00,0.81,0.94,0.942673
296399,The Hollywood Walk of Fame is a series of side...,There are many stars on the sidewalk .,,0.16,0.13,0.20,0.44,0.97,1.00,0.528221
296400,Boulogne-Billancourt is the most populous subu...,"Consequently , Boulogne-Billancourt is one of ...",,0.74,0.81,0.76,0.86,0.97,1.00,0.887207


In [6]:
data_test = data_df[(data_df['similarity'] > 0.9)]

In [7]:
data_test

,original_text,simple_text,original_text_preprocessed,WLR,CLR,LR,DTDR,WRR,LMFMR,similarity
0,There is manuscript evidence that Austen conti...,There is some proof that Austen continued to w...,,0.89,0.91,0.68,0.86,0.92,0.69,0.937799
1,"In a remarkable comparative analysis , Mandaea...",Mandaean scholar Säve-Söderberg showed that Ma...,,0.75,0.71,0.81,1.00,0.99,1.47,0.972845
3,Cogeneration plants are commonly found in dist...,Cogeneration plants are commonly found in dist...,,1.03,1.01,0.98,1.00,1.01,1.11,0.997548
5,When Japan earned another race on the F1 sched...,When Japan was added back to the F1 schedule t...,,1.00,0.95,0.89,1.00,0.94,0.75,0.953728
6,This marked the first motorcycle racing event ...,This was the first motorcycle racing event at ...,,1.00,0.97,0.97,1.00,0.97,1.13,0.990993
...,...,...,...,...,...,...,...,...,...,...
296388,"After Kittanning merged with Wickboro , an est...","After Wickboro became part of Kittanning , an ...",,1.06,1.00,0.72,1.25,0.90,0.07,0.955896
296390,"At 37 metres , it is the tallest lighthouse on...",With 37 metres of height it is the tallest lig...,,1.07,1.14,0.91,1.00,0.99,1.00,0.986154
296394,Le Wast is a commune in the Pas-de-Calais depa...,Le Wast is a commune . It is found in the regi...,,1.44,1.28,0.71,0.86,0.82,1.00,0.970736
296397,These structures form at the high-pressure dep...,These structures form at the high-pressure dep...,,1.05,1.03,0.96,1.20,1.00,1.23,0.993901


In [8]:
from datasets import Dataset

In [9]:
data_pandas = Dataset.from_pandas(data_test)

In [10]:
data_pandas

Dataset({
    features: ['original_text', 'simple_text', 'original_text_preprocessed', 'WLR', 'CLR', 'LR', 'DTDR', 'WRR', 'LMFMR', 'similarity', '__index_level_0__'],
    num_rows: 103321
})

In [11]:
data_pandas = data_pandas.remove_columns('__index_level_0__')

In [12]:
data_pandas

Dataset({
    features: ['original_text', 'simple_text', 'original_text_preprocessed', 'WLR', 'CLR', 'LR', 'DTDR', 'WRR', 'LMFMR', 'similarity'],
    num_rows: 103321
})

In [13]:
data_pandas.save_to_disk('../resources/preprocessed_data/wikilarge_sim0.9')